In [9]:
%matplotlib inline

In [10]:
import pyathena as pa
from pyathena.set_plt import *

In [11]:
import glob

In [37]:
def norm_log_hist_plot(hist,edges,**kwargs):
    if hist.sum() > 0: 
        plt.bar(edges[:-1],hist,np.diff(edges),align='edge',**kwargs)


In [55]:
def plot_sf_mass_func(starfiles):
    for sf in starfiles:
        sp=pa.read_starvtk(sf)
        
        plt.ylim(0.5,100)
        plt.xlim(2,7)
        plt.yscale('log')

        if len(sp) >0:
            if 'flag' in sp:
                cl=sp[(sp.mass > 0) & (sp.flag > -2)]
                cl_young=sp[(sp.mass > 0) & (sp.flag > -2) & (sp.mage*units['Myr'] < 40)]
                cl_ini=sp[(sp.mass > 0) & (sp.flag == -2)]
            else:
                cl=sp[(sp.mass > 0) & (sp.age < sp.time)]
                cl_young=sp[(sp.mass > 0) & (sp.age < sp.time) & (sp.age*units['Myr'] < 40)]
                cl_ini=sp[(sp.mass > 0) & (sp.age > sp.time)]

            cl_hist,cl_edges=np.histogram(np.log10(cl.mass*units['Msun']),range=[2,7],bins=50)
            cl_young_hist,cl_young_edges=np.histogram(np.log10(cl_young.mass*units['Msun']),range=[2,7],bins=50)
            cl_ini_hist,cl_ini_edges=np.histogram(np.log10(cl_ini.mass*units['Msun']),range=[2,7],bins=50)
        
            norm_log_hist_plot(cl_ini_hist,cl_ini_edges,alpha=0.5,label='initial clusters',color='C0')
            norm_log_hist_plot(cl_hist,cl_edges,alpha=0.8,label='all new clusters',color='C1')
            norm_log_hist_plot(cl_young_hist,cl_young_edges,label='new young clusters',color='C2')

            plt.text(6.,10,'t={:6.2f}Myr'.format(sp.time[0]*units['Myr']))
        plt.legend()
        plt.savefig('{}massfunc.png'.format(sf.replace('vtk','')))
        plt.clf()

In [62]:
base='/perseus/scratch/gpfs/changgoo/'
base='/tigress/changgoo/'
dirs=glob.glob('{}/R*metal/starpar'.format(base))
ids=[]
for dd in dirs:
    ids.append(dd.split('/')[-2])
print ids
pid='R2_4pc_noacc_largebox'
ids=['R4_4pc_nograv']

['R4_2pc_metal', 'R16_16pc_metal', 'R2_1pc_metal', 'R8_8pc_metal', 'R2_2pc_metal', 'R4_4pc_metal', 'R2_4pc_metal', 'R8_4pc_metal']


In [63]:
for pid in ids:
    starfiles=glob.glob('{}{}/starpar/{}.*.starpar.vtk'.format(base,pid,pid))
    png_files=glob.glob('{}{}/starpar/{}.*.starpar.massfunc.png'.format(base,pid,pid))
    nf1=len(starfiles)
    nf2=len(png_files)
    starfiles.sort()
    print pid, nf1, nf2
    if nf1 > nf2:
        plot_sf_mass_func(starfiles)

R4_4pc_nograv 401 0


In [48]:
for sf in starfiles:
    sp = pa.read_starvtk(sf)
    print len(sp)

0
0
0
0
0
0
0
0
0
0
0
1
1
2
3
4
5
6
7
10
10
18
20
24
26
33
46
53
58
65
72
76
89
110
128
148
180
217
258
285
323
341
362
386
402
414
441
474
492
511
515
514
522
511
503
496
483
475
456
449
430
412
405
396
393
385
376
355
344
350
327
319
308
305
303
298
296
288
284
279
281
281
283
283
284
286
284
290
296
298
298
290
291
297
295
294
292
287
291
284
286
292
293
297
303
304
311
312
328
325
331
336
347
348
342
339
343
336
336
327
319
310
291
288
277
271
268
260
258
250
258
257
257
266
275
277
265
263
264
270
269
273
280
303
331
357
384
400
421
443
453
473
480
484
495
495
502
507
506
492
484
484
474
465
437
429
413
406
388
378
358
347
326
321
319
312
303
286
281
272
296
309
324
329
346
353
373
381
388
420
442
461
485
500
517
531
535
542
544
557
565
559
558
547
549
550
535
528
516
496
477
459
441
424
400
392
368
352
332
311
302
288
277
266
259
255
250
244
244
252
260
262
265
282
300
306
322
336
350
365
388
424
466
493
533
553
568
590
597
603
600
597
597
599
613
623
626
614
599
590
572
559
548


In [ ]:
outfile=glob.glob('{}{}/*.out'.format(base,pid))

fp=open(outfile[0],'r')
lines=fp.readlines()
fp.close()

tmerge=[]
for i,l in enumerate(lines):
    
    if l.startswith('Merging'):
        splines=lines[i+1].split()[1].split('=')
        print splines
        if splines[0] == 'time': tmerge.append(float(splines[1]))

In [102]:
tmerge

[9.664307,
 10.01605,
 10.2992,
 10.9477,
 11.55533,
 11.90858,
 12.63802,
 13.1504,
 13.44783,
 13.94308,
 14.26238,
 14.52581,
 14.84996,
 15.3236,
 16.0804,
 16.34998,
 18.00005,
 19.80596,
 19.89591,
 22.88839,
 23.99786,
 25.45814,
 25.72566,
 28.34599,
 29.50623,
 30.68624,
 30.96585,
 31.30524,
 31.49122,
 32.23173]